# Лабораторная работа по модулю “Построение моделей”

### 1. Загрузить данные из папки Data Folder.

In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
%pylab inline
%config InlineBackend.figure_format = 'svg' 

from pylab import rcParams
rcParams['figure.figsize'] = 10,10
pd.set_option('display.max_columns', 100)

import warnings
warnings.simplefilter('ignore')

ModuleNotFoundError: No module named 'matplotlib.artist'

In [ ]:
data = pd.read_csv(r'C:\Users\pc\PycharmProjects\Netology\Classwork\data\flag.data')

In [ ]:
data.head()

In [ ]:
afg = data.columns

In [ ]:
col_names = ['country', 'landmass', 'zone', 'area', 'population', 'languag', 'religion', 'bars', 'stripes', 'colours', 'red', 'green', 'blue', 'gold', 'white', 'black', 'orange', 'dominantcolour', 'circles', 'crosses', 'saltires', 'quarters', 'sunstars', 'crescent', 'traingle', 'icon', 'animate', 'text', 'topleftcolour', 'botrightcolor']

In [ ]:
data.columns = col_names

In [ ]:
afg = pd.DataFrame(afg).T
afg.columns = col_names

data = data.append(afg, ignore_index=True)

In [ ]:
data.info()

### 2. Провести описательный анализ предложенных данных

**В наших данных отсутствуют пропуски. Достаточно большое количество категориальных переменных (большинство). Данные чистые и их совсем немного.** 

**Посмотрим на количество стран каждой религии.** 

**religion: 0=Catholic, 1=Other Christian, 2=Muslim, 3=Buddhist, 4=Hindu, 5=Ethnic, 6=Marxist, 7=Others**

In [ ]:
data[['religion', 'country']].groupby('religion').count().plot(kind='barh')

### 3. Предложить стратегию по работе с категориальными переменными

**Для работы с категориальными переменными мы будем использовать get_dummies. Составим массив этих переменных.**

In [ ]:
y = data['religion'].astype(int)
data = data.drop(columns=['religion'])

int_columns = np.array(['area', 'population', 'bars', 'stripes', 'circles', 'crosses', 'saltires', 'quarters', 'sunstars', 'crescent', 'traingle', 'icon', 'animate', 'text'])
obj_columns = np.array([ x for x in data.columns if x not in int_columns ])
data[int_columns] = data[int_columns].astype(float)

In [ ]:
df = pd.get_dummies(data, columns=obj_columns)

In [ ]:
df = df.T.drop_duplicates().T
columns = df.columns

In [ ]:
len(columns)

Мы получили 269 колонок.

### 4. Провести углубленный анализ данных (корреляции переменных,визуализация взаимосвязей)

**Для поиска корреляций в наших данных требуется перевести в тип float те переменные которые являются количественными.** 

In [ ]:
import seaborn as sns
sns.set()
sns.heatmap(data[int_columns].corr(), annot=True, fmt=".2f",
            linewidths=0.5);

**Видно, что есть положительная корреляция между количество населения и площадью, а также, например, между crosses и saltires**

**Можно визуализировать взаимосвязь между популяцией и площадью.**

In [ ]:
plt.scatter(data.area, data.population)
plt.ylim(0, 200)
plt.xlim(0 , 3000)

**Видно, что какой-то очевидной функциональной зависимости нет.**

### 5. Следует ли увеличивать/уменьшать размерность признакового множества (какой подход выдает лучшие результаты)

In [ ]:
df.info()

**В целом у нас мало данных (194 x 269), 407.8 KB, поэтому лучше использовать все данные, чтобы не терять в точности.**

### 6. Следует ли балансировать классы путем oversampling/undersampling? (улучшает ли подход результаты)

In [ ]:
y.hist()

**В целом у нас есть классы, количество которых заметно меньше, чем других (это классы 3, 4, 7). Так что балансировка классов была бы полезна.**

### 7. Решить задачу мультиклассовойклассификации и предсказать религию страны(religion)по деталям ее флага

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2)

In [ ]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, recall_score, accuracy_score, f1_score, precision_score

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'max_depth':range(1, 7, 1),
    'learning_rate':np.arange(0.1, 0.4, 0.1),
}

gsearch1 = GridSearchCV(
            estimator=XGBClassifier(),
            param_grid = parameters,
            scoring='accuracy',
            n_jobs=-1,
            iid=False,
            cv=3
)
gsearch1.fit(X_train, y_train)
gsearch1.best_params_

In [ ]:
clf = XGBClassifier(
   max_depth=gsearch1.best_params_['max_depth'],
   learning_rate=gsearch1.best_params_['learning_rate'],
)


clf.fit(X_train, y_train)

y_test_predict = clf.predict(X_test)

In [ ]:
recall_score(y_test, y_test_predict, average='weighted')

In [ ]:
accuracy_score(y_test, y_test_predict)

In [ ]:
f1_score(y_test, y_test_predict, average='weighted')

In [ ]:
precision_score(y_test, y_test_predict, average='weighted')

### 8.Сделать кросс-валидациюданных с использованием подхода K-fold

Для более адекватной оценкци качества модели проведем кросс-валидацию.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

kf = KFold(n_splits=10)
results = cross_val_score(clf, X_train, y_train, cv=kf)

In [ ]:
results.mean()

In [ ]:
from sklearn.model_selection import StratifiedKFold

stack_cv = StratifiedKFold(n_splits=10)

def cross_val(clf, X_train, y_train, X_test, y_test, stack_cv):
    stacked_test = np.zeros_like(y_test, dtype=float)
    
    for i, (train_ind, test_ind) in enumerate(stack_cv.split(X_train, y_train)):
        clf.fit(X_train.iloc[train_ind], y_train.iloc[train_ind])
        stacked_test += clf.predict(X_test)
    stacked_test = stacked_test / stack_cv.n_splits
    return stacked_test.round(0)

In [ ]:
y_cross = cross_val(clf, X_train, y_train, X_test, y_test, stack_cv)

In [ ]:
recall_score(y_test, y_cross, average='weighted')

In [ ]:
accuracy_score(y_test, y_cross)

In [ ]:
f1_score(y_test, y_cross, average='weighted')

In [ ]:
precision_score(y_test, y_cross, average='weighted')

### 9.Рассчитать Feature Importances для переменных модели.

In [ ]:
IDcol = 'Id'
y_label = 'SalePrice'
predictors = [x for x in columns if x not in [y_label, IDcol]]
plt.figure(figsize=(10,7))
(pd.Series(clf.feature_importances_, predictors).sort_values(ascending=False))[:10].plot(kind='barh')

### 10. Построить Confusion Matrix для ваших результатов.

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_test_predict)